<a href="https://colab.research.google.com/github/Thiziri-Hafir/Donnees_Massives/blob/main/quarto_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "Projet : Changement de production de café en fonction du climat au Brésil"
format:
  html:
    code-fold: true
jupyter: python3
---

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install pyspark

# Importing the data

In [ ]:
import pyspark
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt

SparkContext.setSystemProperty('spark.executor.memory', '8g')
SparkContext.setSystemProperty('spark.driver.memory', '45G')

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

In [ ]:
url = "https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/arabica_data_cleaned.csv"
raw_arabica = pd.read_csv(url)

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

# Auxiliar functions

def count_missings(spark_df,sort=True):
    """
    Counts number of nulls and nans in each column
    """
    df = spark_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)

    return df

def equivalent_type(f):
    """ 
         Function that changes the input Python type to PySpark type
    """
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return DoubleType()
    elif f == 'float32': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    """ 
         Function that define the structure of the input
    """
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    """ 
         Function that transforms an input pandas dataframe to a spark dataframe
    """
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return spark.createDataFrame(pandas_df, p_schema)

In [ ]:
# Creaate a spark dataframe from our initial pandas dataframe 

raw_arabica[['Variety']]= raw_arabica[['Variety']].fillna('unknown')
df_arabica = pandas_to_spark(raw_arabica)  
df_arabica.show(10)

# Basic structural transformations and harmonization

## Renaming columns and selecting the columns and rows of interest

### In the next cell, we rename the columns to facilitate their use.


In [ ]:
tempList = [] #Edit01
for col in df_arabica.columns:
        new_name = col.strip()
        new_name = "".join(new_name.split())
        new_name = new_name.replace('.','_') # EDIT
        tempList.append(new_name) #Edit02
print(tempList) #Just for the sake of it #Edit03

df_arabica = df_arabica.toDF(*tempList) #Edit04

In [ ]:
count_missings(df_arabica)

#### We decided to focus on Brazil as it is one of the biggest coffee producer and presents interesting geoclimatic features and diversity

In [ ]:
df_arabica = df_arabica.filter(df_arabica["Country_of_Origin"]=='Brazil')\
          .distinct()\
          .select(["Farm_name","Altitude","Variety","Aroma","Flavor","Aftertaste",\
                   "Acidity","Body","Balance","Uniformity","Clean_Cup","Sweetness","Total_Cup_Points","altitude_mean_meters"])
df_arabica.show(100, 100)

In [ ]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col

df_arabica = df_arabica.withColumn('Farm_name', regexp_replace(col('Farm_name'), "/", "_"))

In [ ]:
# We drop all rows with where altitude_mean_meters==NaN
df_arabica = df_arabica.na.drop(subset=["altitude_mean_meters"]) 

In [ ]:
df_arabica.show(10,10)

#### At first sight, we can notice that we need to harmonize the Altitude column values and link each Farm_name to its position.


## Harmonizing the altitude column

## Linking Farm_names to actual longitudes and latitudes

 #### We created by hand a dataset with the position of the fields of each Farm_name and performed a simple outer_join

In [ ]:
df_farms = spark.createDataFrame(
    [
        ("fazenda rio verde",-21.877600079894428, -45.17833587173798 ), 
        ("fazenda do lobo", -20.05845833645814, -45.551377369807916),
        ("fazenda grota funda",-4.498676298642221, -46.01438009152326),
        ("sitio claro",-12.404400112670487, -57.0307320652044),
        ("santa alina",-21.76121880758086, -46.674253002625804),
        ("fazenda chamusca",-21.4532483711391, -45.22708818550831),
        ("santa maria",-16.609194745742165, -46.98365752985455),
        ("capoeirinha",-18.64698614414741, -45.796849953979795),
        ("fazenda do sertao",-22.09933939187727, -45.18968007378277),
        ("santa fé 2",-17.582654936016926, -47.2198752950817),
        ("café do paraíso",-22.094568087843687, -45.155496432162685),
        ("cachoeira da grama farm",-21.76626556295618, -46.702161544954144),
        ("são francisco da serra",-22.629913000779446, -44.601043902591805),
        ("fazenda jericó",-18.676315608302055, -45.70281563093488),
        ("sertao farm",-22.099418916815903, -45.18965861611114),
        ("campo das flores",-20.312258124962906, -43.28548394936318),
        ("olhos d'agua",-18.63831446703355, -46.952789277527444),
        ("fazenda serra de três barras",-19.560780988980817, -46.579303497451384),
        ("fazenda pantano",-18.631997524421426, -46.82473127385452),
        ("pereira estate coffee",-22.112480964214626, -45.15508902245408),
        ("rio verde",-21.940510105475326, -45.176192869009476),
        ("sitío são geraldo",-22.59528143237296, -46.66759241371165),
        ("fazenda baipendi",-21.45008093579659, -46.8357381855336),
        ("água limpa",-21.44645605895183, -46.82635044153258),
        ("fazenda kaquend",-21.435236392716707, -46.83244978479159),
        ("fazenda santo antonio",-21.40862617568116, -46.80560442679252),
        ("fazenda vista alegre",-21.44237673402853, -46.818513026788274),
        ("fazenda recreio",-21.780100162105256, -46.67880344388111),
        ("fazenda capoeirnha",-21.761009557830032, -46.67507912298348),
        ("pantano",-21.444638601462618, -46.81747769417689),
        ("fazenda são sebastião",-21.444863288363614, -46.827235570452125),
        ("santa bárbara",-18.52732198399872, -47.569893903127564),
        ("santa mariana",-23.19046310851501, -50.55948683365515),
        ("sertao",-22.095760723175886, -45.189755175672296),
        ("são rafael_ ra/ras certified",-22.784292492565246, -47.032534461666536),
        ("sitío santa luzia",-22.366016295045306, -46.47371725096838),
        ("fazenda são josé mirante",-22.785019797545694, -47.03177457322967),
        ("cianorte",-22.774138613223663, -47.02968245276757),
        ("juliana",-21.46270024286904, -46.832515362350975),
        ("sitío corrego da olaria/são caetano",-22.745464486282955, -47.0338567362583),
        ("fazenda serra negra",-21., -46.674253002625804),
        ("fazendas klem ltda",-20.27591897959779, -41.876604035124465),
        ("castelhana farm",-18.922960557030457, -47.45831617257145),
        ("leticia farm",-19.807319369835636, -42.215697182335234),
        ("helena",-22.107657064931225, -48.32067672583973),
        ("caxambu",-21.339808722609096, -45.42201735838091),
    ],
    ["Farm_name", "lat","long"]  # we add column names here
)

In [ ]:
# Joining the two dataframes to made one 
df_arabica = df_arabica.join(df_farms, on=['Farm_name'], how='left_outer')

In [ ]:
df_arabica.show(10)

### We then filter the coffee to keep only those whose origin is known (Farm_name).

In [ ]:
df_arabica = df_arabica.filter(df_arabica.Farm_name != "NaN")
df_arabica.show(10)

#### We also replace NaN values in the Variety column by "Unknonwn" as the information about the variety is missing.

In [ ]:
df_arabica = df_arabica.replace(float('nan'), None)

df_arabica.show(10)

# Data Exploration and plotting

#### Now that we cleaned and structured the data, we will explore them to fully understand them and get more details abour their behavour. 

## Location of coffee farms 

#### On the map below, we can see the projection of the farms that produce coffee in Brazil. We can see at first sight that they are mainly in the south east of the country, on the side of the Atlantic Ocean. This gives us a first idea of the meteorological conditions necessary for this production,since the south-east is a region with a subtropical climate. The summer can be very hot, even stifling, and there are many tropical rains, as sudden as they can be violent, especially in Rio and São Paulo. Winter, from June to September, is rather mild, with temperatures between 12 and 25 degrees.

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(df_arabica.toPandas(), lat="lat", lon="long", hover_name="Farm_name", hover_data=["altitude_mean_meters",], color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## The correlation between the different taste of coffee variables 

In [ ]:
df_arabica.groupBy('Variety')\
          .count().alias("count")\
          .show()

Using the Seaborn library, we plot the individuals on a dual axis to see if there are overall correlations between the taste variables (aroma, flavor, aftertaste, acidity, sweetness). We can also observe if individuals of the same variety share the same correlation on this axis. Finally we can observe how the varieties are distributed on the correlation axis.

In [ ]:
g = sns.PairGrid(df_arabica.toPandas(), vars=['Aroma','Flavor','Aftertaste','Acidity','Sweetness'],
                 hue='Variety', palette='RdBu_r')
g.map(plt.scatter, alpha=0.4)
g.add_legend()

Alternative visualisation

In [ ]:
#!pip install seaborn --upgrade 
#if there are errors : upgrade seaborn then restart kernel

In [ ]:
g = sns.PairGrid(df_arabica.toPandas(), vars=['Aroma','Flavor','Aftertaste','Acidity','Sweetness'], hue='Variety', palette = "deep")
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot, warn_singular=False)
g.map_diag(sns.kdeplot, warn_singular=False, lw = 3)
g.add_legend()

We may need more individuals to find a clear pattern.

## Taste profile and scoring relation 

### In the section below, we will see how the different tastes of coffee influence the rating of Total_Cup_Score

#### Creating the features vector which are : Aroma, Flavor, Aftertaste, Acidity, Body, Balance, Uniformity, Clean_Cup, Sweetnes.

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ["Aroma","Flavor","Aftertaste",\
                   "Acidity","Body", "Balance", "Clean_Cup", "Uniformity","Sweetness"], outputCol = 'features')
vec_df_arabica = vectorAssembler.transform(df_arabica)

#### Train test splitting

In [ ]:
train_taste,test_taste  = vec_df_arabica.randomSplit([0.7, 0.3])

##### LR model

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='Total_Cup_Points', maxIter=10)
lr_model = lr.fit(train_taste)
print("Coefficients: " + str(lr_model.coefficients[:5])+ "\n"+ str(lr_model.coefficients[5:]))
print("Intercept: " + str(lr_model.intercept))

All taste characteristics are scored out of 10 and the characteristic Total_Cup_Points is their aggregation.

We can therefore assume that all taste characteristics have the same importance.

By running a linear regression model, we can see that all the features have a close importance, but some of them have a slightly greater impact on the model.

We can then make 2 assumptions:

1st assumption: because of the correlation between the characteristics, some of them are considered more important because when their score is high, it increases the score of the others.

2nd assumption: some characteristics have a high variance and the others a low one, those with a high variance will have more impact on the final score.

The slightly more important characteristics seem to be body and balance, but they are not very significant.

In [ ]:
# Evaluation on the train data 
print("MSE: ", lr_model.summary.meanSquaredError)
print("MAE: ", lr_model.summary.meanAbsoluteError)
print("R-squared: ", lr_model.summary.r2)

#### Predicting and evaluating the model

In [ ]:
lr_predictions = lr_model.transform(test_taste)
lr_predictions.select("prediction","Total_Cup_Points","features").show(5)

In [ ]:
# Evaluation on test data 
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Total_Cup_Points")

print("MAE on test data = %g" % lr_evaluator.evaluate(lr_predictions, {lr_evaluator.metricName: "mae"}))
print("MSE on test data = %g" % lr_evaluator.evaluate(lr_predictions, {lr_evaluator.metricName: "mse"}))
print("R-squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions, {lr_evaluator.metricName: "r2"}))


#### Th R squared of our model is very high that means that the variation of tastes variables explains very well the note given for a coffee. In the other hand, our model fits quite well the data.

#### In the graph below, we can see that the prediction values curve is almost superimposed on the original values curve

In [ ]:
x_ax = range(0,lr_predictions.count())
y_pred = lr_predictions.select("prediction").collect()
y_orig = lr_predictions.select("Total_Cup_Points").collect()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(x_ax, y_orig, label="Total_Cup_Points")
plt.plot(x_ax, y_pred, label="prediction")
plt.title("Real notes on the test data and the predicted ones")
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show()  
 


## Variety and scoring relation

#### For this one we will just use visualizations to see the variation of coffee grade according to the variet.

In [ ]:
var = df_arabica.groupBy("Variety").mean("Total_Cup_Points")
var.show()

In [ ]:
px.bar(var.toPandas(), x='Variety', y='avg(Total_Cup_Points)')

In [ ]:
plt.figure(figsize=(15,8))
sns.set_theme(style="darkgrid")
sns.lineplot(x="Variety", y='avg(Total_Cup_Points)', data=var.toPandas()).set_title('Variation of the coffee grade according to the variety')

#### Not a very convincing plot, the Total_Cup_Points are very close (in the 80-90 range) and variety doesn't seem to matter that much

## Farm and scoring relation

In [ ]:
farmr = df_arabica.groupBy("Farm_name").mean("Total_Cup_Points")
farmr.show()

In [ ]:
px.bar(farmr.toPandas().sort_values(by=['avg(Total_Cup_Points)']), x='Farm_name', y='avg(Total_Cup_Points)')

#### Once again the plot is uninteresting, we can try to scale Total_Cup_Points to see more clearly (and with it let's do it on the tastes column since they suffer the same problem)

## Normalizing

In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf

In [ ]:
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())
df_arabica_scaled = df_arabica

for i in ["Aroma","Flavor","Aftertaste",\
                   "Acidity","Body","Balance","Total_Cup_Points"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    df_arabica_scaled = pipeline.fit(df_arabica_scaled).transform(df_arabica_scaled).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")

print("After Scaling :")
df_arabica_scaled.show(5)

source : https://stackoverflow.com/questions/40337744/scalenormalise-a-column-in-spark-dataframe-pyspark

# Back to Exploration

## Let's try again the previous plots

## Variety

In [ ]:
var = df_arabica_scaled.groupBy("Variety").mean("Total_Cup_Points_Scaled")
px.bar(var.toPandas(), x='Variety', y='avg(Total_Cup_Points_Scaled)')

In [ ]:
plt.figure(figsize=(15,8))
sns.set_theme(style="darkgrid")
sns.lineplot(x="Variety", y='avg(Total_Cup_Points_Scaled)', data=var.toPandas()).set_title('Variation of the coffee grade according to the variety')

# TOOOOOOOOOOOOOOOOOOOO DOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

## Farms

In [ ]:
farmr = df_arabica_scaled.groupBy("Farm_name").mean("Total_Cup_Points_Scaled")
px.bar(farmr.toPandas().sort_values(by=['avg(Total_Cup_Points_Scaled)']), x='Farm_name', y='avg(Total_Cup_Points_Scaled)')

### Way clearer

#### We can notice that farms are more important offer more distinction than variety, we will see furthermore

## From now on we will use only the scaled version

In [ ]:
df_arabica = df_arabica_scaled

cols = ("Aroma","Flavor","Aftertaste",\
                   "Acidity","Body","Balance","Total_Cup_Points")
df_arabica = df_arabica.drop(*cols)
for i in ["Aroma","Flavor","Aftertaste",\
                   "Acidity","Body","Balance","Total_Cup_Points"]:
                   df_arabica = df_arabica.withColumnRenamed(i+"_Scaled", i)

df_arabica.show()
df_arabica.printSchema()

## Farms all of fame and their strengths

### Since farms seem interesting we can plot radar plots of the taste they offer (meaned).
#### We will focus on the most renowed farms

In [ ]:
from pyspark.sql.functions import desc
hof = df_arabica.groupBy("Farm_name").mean("Total_Cup_Points").sort(desc("avg(Total_Cup_Points)"))
hof.show(6)

In [ ]:
 li = hof.select("Farm_name").take(3)

In [ ]:
li = ["fazenda kaquend","fazenda recreio","sitío são geraldo","sitio claro","fazenda grota funda"]

In [ ]:
df_hof = df_arabica.filter(df_arabica.Farm_name.isin(li))
df_hof.show()

In [ ]:
import plotly.graph_objects as go
from pyspark.sql.functions import mean

categories = ["Aroma","Flavor","Aftertaste",\
                   "Acidity","Body","Balance"]

df_hof_pd = df_hof.select(["Farm_name","Aroma","Flavor","Aftertaste",\
                   "Acidity","Body","Balance"]).toPandas().set_index('Farm_name')


df_mean_pd = df_arabica.select(mean("Aroma"), mean("Flavor")\
                               , mean("Aftertaste"), mean("Acidity"), mean("Body"), mean("Balance"))\
                               .toPandas()

fig = go.Figure()

for g in df_hof_pd.index:
    fig.add_trace(go.Scatterpolar(
        r = df_hof_pd.loc[g].values,
        theta = categories,
        fill = "toself",
        name = f'{g}'
    ))


fig.add_trace(go.Scatterpolar(
        r = df_mean_pd.values,
        theta = categories,
        fill = "toself",
        name = f'MEAN'
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=False,
      range=[0, 1]
    )),
  showlegend=False
)

fig.show()

# Recommend a coffee a taste profile

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator



assembler = VectorAssembler(inputCols= categories,outputCol="features")

df_km= assembler.transform(df_arabica)

from pyspark.ml.evaluation import ClusteringEvaluator
cluster_score =[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='features', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')

nbmaxclust = 20 
for nbclust in range(2,nbmaxclust):
    
    kmeans=KMeans(featuresCol='features', k=nbclust)
    
    kmeans=kmeans.fit(df_km).transform(df_km)
    
    score = evaluator.evaluate(kmeans)
    
    cluster_score.append(score)
    
    print("Cluster Score:",score)


In [ ]:
score_to_plot = pd.DataFrame({'score': cluster_score, 'nbclust': range(2,nbmaxclust)})

In [ ]:
fig = px.line(score_to_plot, x="nbclust", y="score")
fig.show()

The optimal number of clusters seems to be 10

In [ ]:
nbo_clust = score_to_plot.sort_values('score',ascending=False).head(1)
nbo_clust

In [ ]:
kmeans=KMeans(featuresCol='features', k=int(nbo_clust.nbclust))
    
kmeans=kmeans.fit(df_km).transform(df_km)

In [ ]:
friend = spark.createDataFrame(
    [(6,6,10,5,10,6)],
    ["Aroma","Flavor","Aftertaste","Acidity","Body", "Balance"])

In [ ]:
kmeans.transform(friend)

## The climat of Brazil over the last 22 years

In [ ]:
url = "/content/gdrive/MyDrive/donmass_proj/"
# Brazil's weather stations
stations_brazil = pd.read_csv(url+"stations_.csv",sep=";")

In [ ]:
stations_brazil.head()

In [ ]:
stations_brazil.info()

In [ ]:
# convert pandas dataframe to spark dataFrame
df_stations = pandas_to_spark(stations_brazil) 

In [ ]:
df_stations.printSchema()

In [ ]:

import plotly.graph_objects as go
fig = px.scatter_mapbox(df_stations.toPandas(), lat="lat", lon="lon", hover_name="id_station", hover_data=["lvl",], color_discrete_sequence=["blue"], zoom=3, height=300)
fig.add_trace(go.Scatter(x=df_arabica.toPandas()['lat'], y=df_arabica.toPandas()['long'],
                    mode='markers',
                    name='farms',
                    marker_color='rgba(152, 0, 0, .8)'))
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

####  As most of the farms are located in the southest of brazil, we onely focus on one and analysis its climate changes, this station's id= A530, ie Poços de Caldas 

In [ ]:
# Brazil's weather 22 year history
weather_brazil = pd.read_csv(url+"weather_sum_all.csv")

In [ ]:
weather_brazil.head()

In [ ]:
weather_brazil.info()

In [ ]:
# fill the missing values with the revious one
weather_brazil = weather_brazil.fillna(method='ffill')

In [ ]:
# new datetime column 
weather_brazil['datetime'] = pd.to_datetime(weather_brazil['DATA (YYYY-MM-DD)'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# create a year column
weather_brazil['year'] = weather_brazil['datetime'].dt.year

In [ ]:
# convert pandas to spark
df_weather = pandas_to_spark(weather_brazil)

In [ ]:
# select the station needed
df_southeast = df_weather.filter(df_weather["ESTACAO"]=='A530')
df_southeast .show(100, 100)

#### We create a Year column

In [ ]:
from pyspark.sql.functions import col, to_timestamp, year
# create a year column 
df_weather = df_weather.withColumn('year', year(col('date')))

In [ ]:
df_weather.show(10)

In [ ]:
!pip install rasterio
!pip install pyproj

In [ ]:
import rasterio
import rasterio.plot
import pyproj
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,8))
f, axarr = plt.subplots(2,2)


for i,itif in enumerate( ["avgtemp.tif", "prec.tif", "srac.tif", "vapr.tif"]) :
  with rasterio.open("/content/gdrive/MyDrive/donmass_proj/"+itif) as src:
    img = src.read(1)
    #print(src.crs)
    if(i<2):
      a = 0
    else:
      a = 1
      i -=2 
    pos = src.index(-52.196283130028455,-13.574677798648072) #long lat 
    axarr[a][i].imshow(img[pos[0]-2200:pos[0]+2200,pos[1]-2200:pos[1]+2200]) 

We wanted to run a model on our rasters to predict the right atmospheric conditions to grow coffee. This could be further used to predict on a dataset containing the same features, in the case of our study, on a predicted climate influenced by global warming.
However after much struggle on rasters, and seeing that these struggle were only on the preprocessing part, we realized that we won't be able to implement a species distribution modelling because of the aforementioned problem, and the extreme specificity of these models, that we never encountered, we decided to go to something closer to our knowledge. 
We got the idea of bringing the problem to a image classification task, predicting the possible presence of coffee from image composed of all bands of the rasters. We sampled some (100,100) squares on fields growing coffe and on some close proximity of the fields.
Unfortunately we encountered two main problems making the model completely bad:
  - We need to sampled way more image, which is time consuming. Having such a low number of image makes the training and the ensuing predictions close to a coinflip, while being overly prone to over-fitting.
  - Pre-trained models are based on 3 bands images, making them hard (with my limited skills) to adapt them to our 12 bands images. I had to create my own poor CNN with no transfer learning which makes it very inefficient.

#### Anyway here's the (poorly written) code 

In [ ]:
df_farms_pd = 

In [ ]:
! mkdir /content/drive/MyDrive/donmass_proj/images/coffee/
! mkdir /content/drive/MyDrive/donmass_proj/images/nocoffee/

In [ ]:
for i,itif in enumerate( ["avgtemp.tif", "prec.tif", "srac.tif", "vapr.tif"]) :
  print(itif)
  with rasterio.open("/content/drive/MyDrive/donmass_proj/"+itif) as src:
    img = src.read(1)
    print(img.shape)
    #center = src.index(-52.196283130028455,-13.574677798648072) #long lat 
    for index, row in df_farms_pd.iterrows():
      farm_name = row["Farm_name"]
      pos = src.index(row["long"], row["lat"])
      im = img[pos[0]-50:pos[0]+50,pos[1]-50:pos[1]+50]
      matplotlib.image.imsave("/content/drive/MyDrive/donmass_proj/images/coffee/"  +str(i) + farm_name + ".png", im)
      im = img[pos[0]+100-50:pos[0]+100+50,pos[1]+100-50:pos[1]+100+50]
      matplotlib.image.imsave("/content/drive/MyDrive/donmass_proj/images/nocoffee/"+str(i) + farm_name + ".png", im)
      

In [ ]:
#! rm -r /content/drive/MyDrive/donmass_proj/images/nocoffee/
#! rm -r /content/drive/MyDrive/donmass_proj/images/coffee/

## Merging them into one N-Dimension Image

In [ ]:
! mkdir /content/drive/MyDrive/donmass_proj/images/coffee/inone
! mkdir /content/drive/MyDrive/donmass_proj/images/nocoffee/inone

In [ ]:
import os
from os import listdir

In [ ]:
#! rm -r /content/drive/MyDrive/donmass_proj/images/nocoffee/inone
#! rm -r /content/drive/MyDrive/donmass_proj/images/coffee/inone

In [ ]:
allcoffee = []
for index, row in df_farms_pd.iterrows():
  farm_name = row["Farm_name"]
  allin = [Image.open("/content/drive/MyDrive/donmass_proj/images/coffee/" + str(i) + farm_name + ".png") for i in range(4)]
  allinone = []
  #print(allin) #Nos 4 geotifs en img
  for image in allin:
    r, g, b,t  = image.split() #4 bandes par img dont 1 inutile
    allinone.append(np.array(r))
    allinone.append(np.array(g))
    allinone.append(np.array(b))
    
  allcoffee.append(np.array(allinone).reshape((100,100,12)))
  #print(allinone)

In [ ]:
allnocoffee = []
for index, row in df_farms_pd.iterrows():
  farm_name = row["Farm_name"]
  allin = [Image.open("/content/drive/MyDrive/donmass_proj/images/nocoffee/" + str(i) + farm_name + ".png") for i in range(4)]
  allinone = []
  #print(allin) #Nos 4 geotifs en img
  for image in allin:
    r, g, b,t  = image.split() #4 bandes par img dont 1 inutile
    allinone.append(np.array(r))
    allinone.append(np.array(g))
    allinone.append(np.array(b))
    
  allnocoffee.append(np.array(allinone).reshape((100,100,12)))

In [ ]:
print(np.array(allnocoffee).shape)
print(np.array(allcoffee).shape)

In [ ]:
allnocoffee = [(0,np.array(allnocoffee)[i]) for i in range(len(np.array(allnocoffee)))]
allcoffee = [(1,np.array(allcoffee)[i]) for i in range(len((allcoffee)))]

In [ ]:
all = allcoffee+allnocoffee

In [ ]:
len(all)

### Should train/test split but low number of images let's see how it trains

# CNN

## Models


In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((100,100,12)),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomBrightness([-0.5,0.5]),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.Conv2D(32, kernel_size=1, padding='valid', activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(48, kernel_size=1, padding='valid', activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='sigmoid')

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(100, 100, 12)),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomBrightness([-0.5,0.5]),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(2, activation='sigmoid')
])


## Data Generator

### there might be a problem here also

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, size, data, batch_size=32,  dataaug=True, shuffle=True):
        self.batch_size = batch_size
        self.data = data    
        self.dataaug=dataaug
        self.indices = range(len(data[:][:]))
        self.shuffle = shuffle
        self.on_epoch_end()
        self.size = size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        keys = [self.indices[k] for k in index]
        return self.__get_data(keys)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

            
    def __get_data(self, keys):        
        X = np.zeros((self.batch_size, self.size, self.size, 12))
        y = np.zeros((self.batch_size,1))

        for i, id in enumerate(keys):
          X[i] = self.data[id][1]
          y[i] = self.data[id][0]

        return X,y
                


In [ ]:
BATCH_SIZE = 16
EPOCHS = 10
IMN_EPOCHS = 1000
IMG_SIZE = 100

In [ ]:
train = DataGenerator(data = all, size = IMG_SIZE, batch_size = BATCH_SIZE)

## Trainning 

In [ ]:
model2.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = "accuracy")

In [ ]:
history = model2.fit(train, epochs = EPOCHS)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])

In [ ]:
model2.predict(DataGenerator(data = allcoffee,size = IMG_SIZE,batch_size=1))